# Первичный анализ данных с Pandas

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/adult.data.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


1. Сколько мужчин и женщин (признак sex) представлено в этом наборе данных?

In [2]:
data['sex'].value_counts()

Male      21790
Female    10771
Name: sex, dtype: int64

2. Каков средний возраст (признак age) женщин?

In [4]:
data[data['sex'] == 'Female']['age'].mean()

36.85823043357163

3. Какова доля граждан Германии (признак native-country)?

In [5]:
data['native-country'].value_counts(normalize=True)['Germany']

0.004207487485028101

4-5. Каковы средние значения и среднеквадратичные отклонения возраста тех, 
кто получает более 50K в год (признак salary) и тех, кто получает менее 50K в год?

In [6]:
data.groupby(['salary'])['age'].agg([np.mean, np.std])

,mean,std
salary,,
<=50K,36.783738,14.020088
>50K,44.249841,10.519028


6. Правда ли, что люди, которые получают больше 50k, имеют как минимум высшее образование? 
(признак education – Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters или Doctorate)

In [7]:
pd.crosstab(data['education'], data['salary'])

salary,<=50K,>50K
education,,
10th,871,62
11th,1115,60
12th,400,33
1st-4th,162,6
5th-6th,317,16
7th-8th,606,40
9th,487,27
Assoc-acdm,802,265
Assoc-voc,1021,361


7. Выведите статистику возраста для каждой расы (признак race) и каждого пола. 
Используйте groupby и describe. Найдите таким образом максимальный возраст мужчин расы Amer-Indian-Eskimo.

In [8]:
data[data['sex'] == 'Female'].groupby(['race'])['age'].describe(percentiles=[])

,count,mean,std,min,50%,max
race,,,,,,
Amer-Indian-Eskimo,119.0,37.117647,13.114991,17.0,36.0,80.0
Asian-Pac-Islander,346.0,35.089595,12.300845,17.0,33.0,75.0
Black,1555.0,37.854019,12.637197,17.0,37.0,90.0
Other,109.0,31.678899,11.631599,17.0,29.0,74.0
White,8642.0,36.811618,14.329093,17.0,35.0,90.0


In [9]:
data[data['sex'] == 'Male'].groupby(['race'])['age'].describe(percentiles=[])

,count,mean,std,min,50%,max
race,,,,,,
Amer-Indian-Eskimo,192.0,37.208333,12.049563,17.0,35.0,82.0
Asian-Pac-Islander,693.0,39.073593,12.883944,18.0,37.0,90.0
Black,1569.0,37.682600,12.882612,17.0,36.0,90.0
Other,162.0,34.654321,11.355531,17.0,32.0,77.0
White,19174.0,39.652498,13.436029,17.0,38.0,90.0


8. Среди кого больше доля зарабатывающих много (>50K): среди женатых или холостых мужчин (признак marital-status)? 
Женатыми считаем тех, у кого marital-status начинается с Married (Married-civ-spouse, Married-spouse-absent или Married-AF-spouse), остальных считаем холостыми.

In [10]:
df = data[data['sex'] == 'Male']
pd.crosstab(df['marital-status'], df['salary']) 

salary,<=50K,>50K
marital-status,,
Divorced,1487,284
Married-AF-spouse,5,4
Married-civ-spouse,7381,5938
Married-spouse-absent,190,23
Never-married,5591,325
Separated,345,49
Widowed,129,39


9. Какое максимальное число часов человек работает в неделю (признак hours-per-week)? 
Сколько людей работают такое количество часов и каков среди них процент зарабатывающих много?

In [11]:
data['hours-per-week'].max()

99

In [12]:
data[data['hours-per-week'] == data['hours-per-week'].max()]['age'].count()

85

In [13]:
pd.crosstab(data['hours-per-week'] == data['hours-per-week'].max(), data['salary'], normalize=True)

salary,<=50K,>50K
hours-per-week,,
False,0.757348,0.240042
True,0.001843,0.000768


10. Посчитайте среднее время работы (hours-per-week) зарабатывающих мало и много (salary) для каждой страны (native-country).

In [14]:
countries = set(data['native-country'])
for country in countries:
    result1 = data[(data['salary'] == '>50K') & (data['native-country'] == country)]['hours-per-week'].mean()
    result2 = data[(data['salary'] == '<=50K') & (data['native-country'] == country)]['hours-per-week'].mean()
    print(country, '>50K', result1, '<=50K', result2)

Peru >50K 40.0 <=50K 35.06896551724138
Canada >50K 45.64102564102564 <=50K 37.91463414634146
China >50K 38.9 <=50K 37.38181818181818
Jamaica >50K 41.1 <=50K 38.23943661971831
Taiwan >50K 46.8 <=50K 33.774193548387096
Philippines >50K 43.032786885245905 <=50K 38.065693430656935
Honduras >50K 60.0 <=50K 34.333333333333336
Trinadad&Tobago >50K 40.0 <=50K 37.05882352941177
Holand-Netherlands >50K nan <=50K 40.0
Ireland >50K 48.0 <=50K 40.94736842105263
Thailand >50K 58.333333333333336 <=50K 42.86666666666667
Yugoslavia >50K 49.5 <=50K 41.6
South >50K 51.4375 <=50K 40.15625
Nicaragua >50K 37.5 <=50K 36.09375
United-States >50K 45.50536884674383 <=50K 38.79912723305605
Cuba >50K 42.44 <=50K 37.98571428571429
Greece >50K 50.625 <=50K 41.80952380952381
El-Salvador >50K 45.0 <=50K 36.03092783505155
Hong >50K 45.0 <=50K 39.142857142857146
Hungary >50K 50.0 <=50K 31.3
France >50K 50.75 <=50K 41.05882352941177
England >50K 44.53333333333333 <=50K 40.483333333333334
Outlying-US(Guam-USVI-etc) >50K 